# VAE Approach

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from ocpa.objects.log.importer.ocel import factory as ocel_import_factory
from ocpa.algo.discovery.ocpn import algorithm as ocpn_discovery_factory
from src.utils import get_happy_path_log, create_flower_model, generate_variant_model, sample_traces, process_log
from ocpa.objects.log.importer.csv import factory as ocel_import_factory_csv
from models.VAE_measure import get_text_data, decode_sequence, create_lstm_vae, VAE_generalization, create_VAE_input
from tqdm import tqdm
import numpy as np

# Order Process

In [4]:
filename = "../src/data/jsonocel/order_process.jsonocel"
ocel = ocel_import_factory.apply(filename)
ocpn = ocpn_discovery_factory.apply(ocel, parameters={"debug": False})
train_log = sample_traces(ocel, ocpn, 10000, save_path='../src/data/playout/ocpn_data_order.txt')


Generate the traces: 100%|██████████| 10000/10000 [00:01<00:00, 7105.36it/s]


In [7]:
train_log

[['Fuel Car',
  'Place Order',
  'Confirm Order',
  'Payment Reminder',
  'Pick Item',
  'Payment Reminder',
  'Payment Reminder',
  'Payment Reminder',
  'Payment Reminder',
  'Load Cargo',
  'Payment Reminder',
  'Payment Reminder',
  'Payment Reminder',
  'Item out of stock',
  'Start Route',
  'End Route',
  'Payment Reminder',
  'Pay Order'],
 ['Fuel Car',
  'Place Order',
  'Confirm Order',
  'Payment Reminder',
  'Pick Item',
  'Payment Reminder',
  'Load Cargo',
  'Item out of stock',
  'Pay Order',
  'Reorder Item',
  'Start Route',
  'End Route'],
 ['Fuel Car',
  'Place Order',
  'Confirm Order',
  'Item out of stock',
  'Reorder Item',
  'Pick Item',
  'Payment Reminder',
  'Pay Order',
  'Load Cargo',
  'Start Route',
  'End Route'],
 ['Fuel Car',
  'Place Order',
  'Confirm Order',
  'Item out of stock',
  'Pick Item',
  'Load Cargo',
  'Start Route',
  'End Route',
  'Pay Order'],
 ['Place Order',
  'Fuel Car',
  'Confirm Order',
  'Item out of stock',
  'Reorder Item',
 

In [8]:
timesteps_max, enc_tokens, characters, char2id, id2char, x, x_decoder = get_text_data(num_samples=10000,
                                                                                      data_path='../src/data/playout/ocpn_data_order.txt')

print(x.shape, "Creating model...")

Number of samples: 10000
Number of unique input tokens: 21
Max sequence length for inputs: 50
(10000, 50, 21) Creating model...


In [9]:
input_dim, timesteps = x.shape[-1], x.shape[-2]
batch_size, latent_dim = 1, 191
intermediate_dim, epochs = 353, 20

vae, enc, gen, stepper = create_lstm_vae(input_dim,
                                         batch_size=batch_size,
                                         intermediate_dim=intermediate_dim,
                                         latent_dim=latent_dim,
                                        )
print("Training model...")

vae.fit([x, x_decoder], x, epochs=epochs, verbose=1)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 21)]   0           []                               
                                                                                                  
 lstm (LSTM)                    (None, 353)          529500      ['input_1[0][0]']                
                                                                                                  
 dense (Dense)                  (None, 191)          67614       ['lstm[0][0]']                   
                                                                                                  
 dense_1 (Dense)                (None, 191)          67614       ['lstm[0][0]']                   
                                                                                              

In [50]:
print("Fitted, predicting...")
#rearrange the input data and get the max amount of characters
input_data = [' '.join(inner_list) for inner_list in train_log]
max_length = max(len(string) for string in input_data)

def decode(s):
    return decode_sequence(s, gen, stepper, input_dim, char2id, id2char, max_length)

log = []

for _ in tqdm(range(500), desc="Sample Traces"):

    id_from = np.random.randint(0, x.shape[0] - 1)

    m_from, std_from = enc.predict([[x[id_from]]])

    seq_from = np.random.normal(size=(latent_dim,))
    seq_from = m_from + std_from * seq_from

    #print(decode(seq_from))
    log.append([decode(seq_from)])

Fitted, predicting...


Sample Traces: 100%|██████████| 500/500 [00:23<00:00, 20.96it/s]


In [51]:
log

[['place order fuel car confirm order payment reminder pay order item out of stock pick item load cargo reorder item pick item start route load cargo start route end route <end> '],
 ['place order confirm order item out of stock payment reminder reorder item pay order fuel car pick item load cargo start route end route <end> '],
 ['place order confirm order item out of stock payment reminder pay order reorder item pick item fuel car load cargo start route end route <end> '],
 ['place order confirm order fuel car item out of stock reorder item pick item pay order load cargo start route end route <end> '],
 ['fuel car place order confirm order pick item load cargo item out of stock pay order reorder item start route pick item end route <end> '],
 ['fuel car place order confirm order payment reminder pick item pay order item out of stock load cargo reorder item pick item start route load cargo end route <end> '],
 ['place order confirm order fuel car item out of stock reorder item pick it

In [54]:
df_log = process_log(log, ocel, ocpn, '../src/data/VAE_generated/order_process_sampled.csv')

In [55]:
df_log

,event_id,event_activity,event_execution,event_timestamp,order,item,delivery
0,0,Place Order,1,2022-01-01 12:39:36.057346,[order1],[item1],[]
1,1,Fuel Car,1,2022-01-01 12:40:36.057346,[],[],[delivery1]
2,2,Confirm Order,1,2022-01-01 12:41:36.057346,[order1],[item1],[]
3,3,Payment Reminder,1,2022-01-01 12:42:36.057346,[order1],[],[]
4,4,Pay Order,1,2022-01-01 12:43:36.057346,[order1],[],[]
...,...,...,...,...,...,...,...
5701,5701,Reorder Item,500,2022-02-27 10:34:49.176835,[],[item500],[]
5702,5702,Pick Item,500,2022-02-27 10:35:49.176835,[],[item500],[]
5703,5703,Load Cargo,500,2022-02-27 10:36:49.176835,[],[item500],[delivery500]
5704,5704,Start Route,500,2022-02-27 10:37:49.176835,[],[item500],[delivery500]


In [5]:
object_types = ["order","item","delivery"]
parameters = {"obj_names": object_types,
              "val_names": [],
              "act_name": "event_activity",
              "time_name": "event_timestamp",
              "sep": ","}
ocel_gen = ocel_import_factory_csv.apply(file_path='../src/data/VAE_generated/order_process_sampled.csv', parameters=parameters)

# OCPN Model

In [5]:
generalization = VAE_generalization(ocel_gen, ocpn)

Precision of IM-discovered net:  0.7413
Fitness of IM-discovered net:  0.7104
VAE Generalization= 0.7255


# Happy Path Order

In [6]:
happy_path__ocel = get_happy_path_log(filename)

In [7]:
happy_path_ocpn = ocpn_discovery_factory.apply(happy_path__ocel, parameters={"debug": False})

In [8]:
happy_path__ocel = get_happy_path_log(filename)
generalization = VAE_generalization(ocel_gen, happy_path_ocpn)

Precision of IM-discovered net:  0.9933
Fitness of IM-discovered net:  0.2669
VAE Generalization= 0.4208


# Flower Model Order

In [9]:
filename = "../src/data/jsonocel/order_process.jsonocel"
ots = ["order","item","delivery"]
flower_ocpn = create_flower_model(filename,ots)

In [10]:
generalization = VAE_generalization(ocel_gen, flower_ocpn)

Precision of IM-discovered net:  0.3011
Fitness of IM-discovered net:  1.0
VAE Generalization= 0.4628


# Variant OCPN

In [7]:
filename = "../src/data/jsonocel/order_process.jsonocel"
ots = ["order","item","delivery"]
ocel = ocel_import_factory.apply(filename)
variant_ocpn = generate_variant_model(ocel,save_path_logs='../src/data/csv/order_process_variants/order_process_variant',object_types = ots,save_path_visuals=f"../reports/figures/order_variant_total.svg" )

Processing Variant Nets: 100%|██████████| 12/12 [00:00<00:00, 6003.30it/s]


#########Start generating Object-Centric Petri Net#########
#########Finished generating Object-Centric Petri Net#########


In [13]:
for transition in variant_ocpn.transitions:
    split_string = transition.name.split("_")
    transition.name = split_string[0]

In [16]:
generalization = VAE_generalization(ocel_gen, variant_ocpn)

Precision of IM-discovered net:  0.7122
Fitness of IM-discovered net:  0.445
VAE Generalization= 0.5477


# P2P Process

In [23]:
filename = "../src/data/jsonocel/p2p-normal.jsonocel"
ocel = ocel_import_factory.apply(filename)
ocpn = ocpn_discovery_factory.apply(ocel, parameters={"debug": False})
train_log = sample_traces(ocel, ocpn, 10000, save_path='../src/data/playout/ocpn_data_p2p.txt')


Generate the traces: 100%|██████████| 10000/10000 [00:01<00:00, 8630.07it/s]


In [12]:
#train_log = create_VAE_input(ocel,'../src/data/VAE_input/order_process.txt')

In [13]:
#train_log

In [14]:
timesteps_max, enc_tokens, characters, char2id, id2char, x, x_decoder = get_text_data(num_samples=10000,
                                                                                      data_path='../src/data/playout/ocpn_data_p2p.txt')

print(x.shape, "Creating model...")

Number of samples: 10000
Number of unique input tokens: 15
Max sequence length for inputs: 24
(10000, 24, 15) Creating model...


In [15]:
input_dim, timesteps = x.shape[-1], x.shape[-2]
batch_size, latent_dim = 1, 191
intermediate_dim, epochs = 353, 20

vae, enc, gen, stepper = create_lstm_vae(input_dim,
                                         batch_size=batch_size,
                                         intermediate_dim=intermediate_dim,
                                         latent_dim=latent_dim,
                                        )
print("Training model...")

vae.fit([x, x_decoder], x, epochs=epochs, verbose=1)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 15)]   0           []                               
                                                                                                  
 lstm (LSTM)                    (None, 353)          521028      ['input_1[0][0]']                
                                                                                                  
 dense (Dense)                  (None, 191)          67614       ['lstm[0][0]']                   
                                                                                                  
 dense_1 (Dense)                (None, 191)          67614       ['lstm[0][0]']                   
                                                                                              

In [16]:
print("Fitted, predicting...")
#rearrange the input data and get the max amount of characters
input_data = [' '.join(inner_list) for inner_list in train_log]
max_length = max(len(string) for string in input_data)

def decode(s):
    return decode_sequence(s, gen, stepper, input_dim, char2id, id2char, max_length)

log = []

for _ in tqdm(range(500), desc="Sample Traces"):

    id_from = np.random.randint(0, x.shape[0] - 1)

    m_from, std_from = enc.predict([[x[id_from]]])

    seq_from = np.random.normal(size=(latent_dim,))
    seq_from = m_from + std_from * seq_from

    #print(decode(seq_from))
    log.append([decode(seq_from)])

Fitted, predicting...


Sample Traces: 100%|██████████| 500/500 [00:20<00:00, 24.46it/s]


In [17]:
log

[['create purchase requisition create purchase order receive goods issue goods receipt verify material receive invoice clear invoice plan goods issue goods issue '],
 ['create purchase requisition create purchase order receive goods issue goods receipt plan goods issue receive invoice clear invoice verify material goods issue '],
 ['create purchase requisition create purchase order receive goods issue goods receipt receive invoice verify material clear invoice plan goods issue goods issue '],
 ['create purchase requisition create purchase order receive goods issue goods receipt receive invoice plan goods issue verify material goods issue clear invoice '],
 ['create purchase requisition create purchase order receive goods issue goods receipt plan goods issue verify material receive invoice clear invoice goods issue '],
 ['create purchase requisition create purchase order receive goods issue goods receipt plan goods issue verify material receive invoice clear invoice goods issue '],
 ['c

In [18]:
df_log = process_log(log, ocel, ocpn, '../src/data/VAE_generated/p2p_process_sampled.csv')

In [19]:
df_log

,event_id,event_activity,event_execution,event_timestamp,INVOICE,GDSRCPT,PURCHREQ,MATERIAL,PURCHORD
0,0,Create Purchase Requisition,1,2022-01-01 13:18:50.341361,[],[],[PURCHREQ1],[MATERIAL1],[]
1,1,Create Purchase Order,1,2022-01-01 13:19:50.341361,[],[],[PURCHREQ1],[MATERIAL1],[PURCHORD1]
2,2,Receive Goods,1,2022-01-01 13:20:50.341361,[],[GDSRCPT1],[],[MATERIAL1],[PURCHORD1]
3,3,Issue Goods Receipt,1,2022-01-01 13:21:50.341361,[],[GDSRCPT1],[],[MATERIAL1],[PURCHORD1]
4,4,Verify Material,1,2022-01-01 13:22:50.341361,[],[],[],[MATERIAL1],[]
...,...,...,...,...,...,...,...,...,...
4495,4495,Plan Goods Issue,500,2022-02-15 08:53:29.737267,[],[],[],[MATERIAL500],[]
4496,4496,Receive Invoice,500,2022-02-15 08:54:29.737267,[INVOICE500],[],[],[],[PURCHORD500]
4497,4497,Verify Material,500,2022-02-15 08:55:29.737267,[],[],[],[MATERIAL500],[]
4498,4498,Clear Invoice,500,2022-02-15 08:56:29.737267,[INVOICE500],[GDSRCPT500],[],[],[PURCHORD500]


In [24]:
object_types = ["PURCHORD","INVOICE","PURCHREQ","MATERIAL","GDSRCPT"]
parameters = {"obj_names": object_types,
              "val_names": [],
              "act_name": "event_activity",
              "time_name": "event_timestamp",
              "sep": ","}
ocel_gen = ocel_import_factory_csv.apply(file_path='../src/data/VAE_generated/p2p_process_sampled.csv', parameters=parameters)

# OCPN Model

In [21]:
generalization = VAE_generalization(ocel_gen, ocpn)

Precision of IM-discovered net:  0.8519
Fitness of IM-discovered net:  1.0
VAE Generalization= 0.92


# Happy Path Order

In [22]:
happy_path__ocel = get_happy_path_log(filename)

In [23]:
happy_path_ocpn = ocpn_discovery_factory.apply(happy_path__ocel, parameters={"debug": False})

In [24]:
happy_path__ocel = get_happy_path_log(filename)
generalization = VAE_generalization(ocel_gen, happy_path_ocpn)

Precision of IM-discovered net:  0.8721
Fitness of IM-discovered net:  0.8129
VAE Generalization= 0.8414


# Flower Model Order

In [25]:
filename = "../src/data/jsonocel/p2p-normal.jsonocel"
ots = ["PURCHORD","INVOICE","PURCHREQ","MATERIAL","GDSRCPT"]
flower_ocpn = create_flower_model(filename,ots)

In [26]:
generalization = VAE_generalization(ocel_gen, flower_ocpn)

Precision of IM-discovered net:  0.1699
Fitness of IM-discovered net:  1.0
VAE Generalization= 0.2905


# Variant OCPN

In [25]:
filename = "../src/data/jsonocel/p2p-normal.jsonocel"
ots = ["PURCHORD","INVOICE","PURCHREQ","MATERIAL","GDSRCPT"]
ocel = ocel_import_factory.apply(filename)
variant_ocpn = generate_variant_model(ocel,save_path_logs='../src/data/csv/p2p-normal_variants/p2p-normal_variant',object_types = ots ,save_path_visuals=f"../reports/figures/p2p_variant_total.svg" )

Processing Variant Nets: 100%|██████████| 20/20 [00:00<00:00, 1913.68it/s]


#########Start generating Object-Centric Petri Net#########
#########Finished generating Object-Centric Petri Net#########


In [26]:
for transition in variant_ocpn.transitions:
    split_string = transition.name.split("_")
    transition.name = split_string[0]

In [27]:
generalization = VAE_generalization(ocel_gen, variant_ocpn)

Precision of IM-discovered net:  0.5204
Fitness of IM-discovered net:  0.8538
VAE Generalization= 0.6466


# BPI Challenge

In [6]:
filename = "../src/data/jsonocel/BPI2017-Final.jsonocel"
ocel = ocel_import_factory.apply(filename)
ocpn = ocpn_discovery_factory.apply(ocel, parameters={"debug": False})
train_log = sample_traces(ocel, ocpn, 10000, save_path='../src/data/playout/ocpn_data_BPI.txt')

Generate the traces: 100%|██████████| 10000/10000 [00:00<00:00, 37878.69it/s]


In [3]:
timesteps_max, enc_tokens, characters, char2id, id2char, x, x_decoder = get_text_data(num_samples=10000,
                                                                                      data_path='../src/data/playout/ocpn_data_BPI.txt')

print(x.shape, "Creating model...")

Number of samples: 10000
Number of unique input tokens: 22
Max sequence length for inputs: 57
(10000, 57, 22) Creating model...


In [4]:
input_dim, timesteps = x.shape[-1], x.shape[-2]
batch_size, latent_dim = 1, 191
intermediate_dim, epochs = 353, 20

vae, enc, gen, stepper = create_lstm_vae(input_dim,
                                         batch_size=batch_size,
                                         intermediate_dim=intermediate_dim,
                                         latent_dim=latent_dim,
                                        )
print("Training model...")

vae.fit([x, x_decoder], x, epochs=epochs, verbose=1)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 22)]   0           []                               
                                                                                                  
 lstm (LSTM)                    (None, 353)          530912      ['input_1[0][0]']                
                                                                                                  
 dense (Dense)                  (None, 191)          67614       ['lstm[0][0]']                   
                                                                                                  
 dense_1 (Dense)                (None, 191)          67614       ['lstm[0][0]']                   
                                                                                              

In [7]:
print("Fitted, predicting...")
#rearrange the input data and get the max amount of characters
input_data = [' '.join(inner_list) for inner_list in train_log]
max_length = max(len(string) for string in input_data)

def decode(s):
    return decode_sequence(s, gen, stepper, input_dim, char2id, id2char, max_length)

log = []

for _ in tqdm(range(500), desc="Sample Traces"):

    id_from = np.random.randint(0, x.shape[0] - 1)

    m_from, std_from = enc.predict([[x[id_from]]])

    seq_from = np.random.normal(size=(latent_dim,))
    seq_from = m_from + std_from * seq_from

    #print(decode(seq_from))
    log.append([decode(seq_from)])

Fitted, predicting...


Sample Traces: 100%|██████████| 500/500 [00:12<00:00, 38.65it/s]


In [8]:
log

[['create application submit assess potential fraud call incomplete files call incomplete files call incomplete files call incomplete files call incomplete files call incomplete files deny accept <end> '],
 ['create application submit assess potential fraud call incomplete files call incomplete files call incomplete files call incomplete files call incomplete files deny call accept <end> '],
 ['create application submit assess potential fraud call incomplete files accept <end> '],
 ['create application submit assess potential fraud call incomplete files call incomplete files call incomplete files call incomplete files call incomplete files accept <end> '],
 ['create application submit assess potential fraud call incomplete files deny accept <end> '],
 ['create application submit accept <end> '],
 ['create application submit assess potential fraud deny accept <end> '],
 ['create application submit accept <end> '],
 ['create application submit accept <end> '],
 ['create application submi

In [9]:
df_log = process_log(log, ocel, ocpn, '../src/data/VAE_generated/BPI_process_sampled_function.csv')

In [10]:
df_log

,event_id,event_activity,event_execution,event_timestamp,application,offer
0,0,Create application,1,2022-01-01 05:42:44.561989,[application1],[]
1,1,Submit,1,2022-01-01 05:43:44.561989,[application1],[]
2,2,Assess potential fraud,1,2022-01-01 05:44:44.561989,[application1],[]
3,3,Call,1,2022-01-01 05:45:44.561989,[application1],[offer1]
4,4,Call,1,2022-01-01 05:46:44.561989,[application1],[offer1]
...,...,...,...,...,...,...
2734,2734,Deny,499,2022-02-09 04:59:04.988919,[application499],[offer499]
2735,2735,Accept,499,2022-02-09 05:00:04.988919,[application499],[]
2736,2736,Create application,500,2022-02-09 03:19:06.599203,[application500],NaN
2737,2737,Submit,500,2022-02-09 03:20:06.599203,[application500],NaN


In [13]:
object_types = ["application","offer"]
parameters = {"obj_names": object_types,
              "val_names": [],
              "act_name": "event_activity",
              "time_name": "event_timestamp",
              "sep": ","}
ocel_gen = ocel_import_factory_csv.apply(file_path='../src/data/VAE_generated/BPI_process_sampled_function.csv', parameters=parameters)

# OCPN Model

In [14]:
generalization = VAE_generalization(ocel_gen, ocpn)

Precision of IM-discovered net:  0.5095
Fitness of IM-discovered net:  0.5666
VAE Generalization= 0.5366


# Happy Path

In [15]:
happy_path__ocel = get_happy_path_log(filename)

In [16]:
happy_path_ocpn = ocpn_discovery_factory.apply(happy_path__ocel, parameters={"debug": False})

In [18]:
generalization = VAE_generalization(ocel_gen, happy_path_ocpn)

Precision of IM-discovered net:  1.0
Fitness of IM-discovered net:  0.4564
VAE Generalization= 0.6267


# Flower Model 

In [19]:
filename = "../src/data/jsonocel/BPI2017-Final.jsonocel"
ots = ["application","offer"]
flower_ocpn = create_flower_model(filename,ots)

In [20]:
generalization = VAE_generalization(ocel_gen, flower_ocpn)

Precision of IM-discovered net:  0.1213
Fitness of IM-discovered net:  1.0
VAE Generalization= 0.2164


# Variant OCPN

In [22]:
import pickle

In [23]:
with open("../src/data/csv/bpi_variant_ocpn.pickle", "rb") as file:
    variant_ocpn = pickle.load(file)

In [24]:
for transition in variant_ocpn.transitions:
    split_string = transition.name.split("_")
    transition.name = split_string[0]

In [25]:
generalization = VAE_generalization(ocel_gen, variant_ocpn)

Precision of IM-discovered net:  1.0
Fitness of IM-discovered net:  0.1825
VAE Generalization= 0.3087


# DS3 Log

In [17]:
filename = "../src/data/jsonocel/DS3.jsonocel"
ocel = ocel_import_factory.apply(filename)
ots = ["incident","customer"]
ocpn = create_flower_model(filename,ots)
#ocpn = ocpn_discovery_factory.apply(ocel, parameters={"debug": False})
train_log = sample_traces(ocel, ocpn, 10000, save_path='../src/data/playout/ocpn_data_ds3.txt')


In [4]:
timesteps_max, enc_tokens, characters, char2id, id2char, x, x_decoder = get_text_data(num_samples=10000,
                                                                                      data_path='../src/data/playout/ocpn_data_ds3.txt')

print(x.shape, "Creating model...")

Number of samples: 10000
Number of unique input tokens: 13
Max sequence length for inputs: 110
(10000, 110, 13) Creating model...


In [5]:
input_dim, timesteps = x.shape[-1], x.shape[-2]
batch_size, latent_dim = 1, 191
intermediate_dim, epochs = 353, 20

vae, enc, gen, stepper = create_lstm_vae(input_dim,
                                         batch_size=batch_size,
                                         intermediate_dim=intermediate_dim,
                                         latent_dim=latent_dim,
                                        )
print("Training model...")

vae.fit([x, x_decoder], x, epochs=epochs, verbose=1)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 13)]   0           []                               
                                                                                                  
 lstm (LSTM)                    (None, 353)          518204      ['input_1[0][0]']                
                                                                                                  
 dense (Dense)                  (None, 191)          67614       ['lstm[0][0]']                   
                                                                                                  
 dense_1 (Dense)                (None, 191)          67614       ['lstm[0][0]']                   
                                                                                              

In [13]:
from nltk.tokenize import word_tokenize

# vectorize the data
input_texts = []
input_characters = set(["\t"])

with open('../src/data/playout/ocpn_data_ds3.txt', "r", encoding="utf-8") as f:
    lines = f.read().lower().split("\n")

for line in lines[: min(10000, len(lines) - 1)]:

    #input_text, _ = line.split("\t")
    input_text = word_tokenize(line)
    input_text.append("<end>")

    input_texts.append(input_text)


In [26]:
print("Fitted, predicting...")
#rearrange the input data and get the max amount of characters
#input_data = [' '.join(inner_list) for inner_list in train_log]

input_data = [' '.join(inner_list) for inner_list in input_texts]
max_length = max(len(string) for string in input_data)*2

def decode(s):
    return decode_sequence(s, gen, stepper, input_dim, char2id, id2char, max_length)

log = []

for _ in tqdm(range(500), desc="Sample Traces"):

    id_from = np.random.randint(0, x.shape[0] - 1)

    m_from, std_from = enc.predict([[x[id_from]]])

    seq_from = np.random.normal(size=(latent_dim,))
    seq_from = m_from + std_from * seq_from

    #print(decode(seq_from))
    log.append([decode(seq_from)])

Fitted, predicting...


Sample Traces: 100%|██████████| 500/500 [01:14<00:00,  6.73it/s]


In [28]:
df_log = process_log(log, ocel, ocpn, '../src/data/VAE_generated/DS3_process_sampled_function.csv')

In [29]:
df_log

,event_id,event_activity,event_execution,event_timestamp,incident
0,0,Awaiting Evidence,1,2022-01-01 06:34:34.868090,[incident1]
1,1,Awaiting Evidence,1,2022-01-01 06:35:34.868090,[incident1]
2,2,Awaiting Evidence,1,2022-01-01 06:36:34.868090,[incident1]
3,3,Awaiting Evidence,1,2022-01-01 06:37:34.868090,[incident1]
4,4,Awaiting Evidence,1,2022-01-01 06:38:34.868090,[incident1]
...,...,...,...,...,...
25187,25187,Awaiting Problem,500,2022-09-10 04:45:10.486805,[incident500]
25188,25188,Active,500,2022-09-10 04:46:10.486805,[incident500]
25189,25189,Awaiting Evidence,500,2022-09-10 04:47:10.486805,[incident500]
25190,25190,Awaiting Evidence,500,2022-09-10 04:48:10.486805,[incident500]


In [31]:
object_types = ["incident"] #does not sample anything with customer object type
parameters = {"obj_names": object_types,
              "val_names": [],
              "act_name": "event_activity",
              "time_name": "event_timestamp",
              "sep": ","}
ocel_gen = ocel_import_factory_csv.apply(file_path='../src/data/VAE_generated/DS3_process_sampled_function.csv', parameters=parameters)

# OCPN Model

In [32]:
generalization = VAE_generalization(ocel_gen, ocpn)

Precision of IM-discovered net:  0.1968
Fitness of IM-discovered net:  1.0
VAE Generalization= 0.3289


# Happy Path Order

In [33]:
happy_path__ocel = get_happy_path_log(filename)

In [34]:
happy_path_ocpn = ocpn_discovery_factory.apply(happy_path__ocel, parameters={"debug": False})

In [35]:
happy_path__ocel = get_happy_path_log(filename)
generalization = VAE_generalization(ocel_gen, happy_path_ocpn)

Precision of IM-discovered net:  0
Fitness of IM-discovered net:  0.0


ZeroDivisionError: float division by zero

# Flower Model Order

In [36]:
filename = "../src/data/jsonocel/DS3.jsonocel"
ots = ["incident","customer"]
flower_ocpn = create_flower_model(filename,ots)

In [37]:
generalization = VAE_generalization(ocel_gen, flower_ocpn)

Precision of IM-discovered net:  0.1968
Fitness of IM-discovered net:  1.0
VAE Generalization= 0.3289


# Variant OCPN

In [39]:
import pickle

In [40]:
with open("../src/data/csv/DS3_variant_ocpn.pickle", "rb") as file:
    variant_ocpn = pickle.load(file)

In [41]:
for transition in variant_ocpn.transitions:
    split_string = transition.name.split("_")
    transition.name = split_string[0]

In [ ]:
generalization = VAE_generalization(ocel_gen, variant_ocpn)

# DS4 Log

In [ ]:
filename = "../src/data/jsonocel/DS4.jsonocel"
ocel = ocel_import_factory.apply(filename)
ocpn = ocpn_discovery_factory.apply(ocel, parameters={"debug": False})
train_log = sample_traces(ocel, ocpn, 10000, save_path='../src/data/playout/ocpn_data_ds4.txt')


In [ ]:
timesteps_max, enc_tokens, characters, char2id, id2char, x, x_decoder = get_text_data(num_samples=2000,
                                                                                      data_path='../src/data/playout/ocpn_data_ds4.txt')

print(x.shape, "Creating model...")

In [ ]:
input_dim, timesteps = x.shape[-1], x.shape[-2]
batch_size, latent_dim = 32, 191
intermediate_dim, epochs = 353, 8

vae, enc, gen, stepper = create_lstm_vae(input_dim,
                                         batch_size=batch_size,
                                         intermediate_dim=intermediate_dim,
                                         latent_dim=latent_dim,
                                        )
print("Training model...")

vae.fit([x, x_decoder], x, epochs=epochs, verbose=1)

In [ ]:
print("Fitted, predicting...")
#rearrange the input data and get the max amount of characters
input_data = [' '.join(inner_list) for inner_list in train_log]
max_length = max(len(string) for string in input_data)

def decode(s):
    return decode_sequence(s, gen, stepper, input_dim, char2id, id2char, max_length)

log = []

for _ in tqdm(range(500), desc="Sample Traces"):

    id_from = np.random.randint(0, x.shape[0] - 1)

    m_from, std_from = enc.predict([[x[id_from]]])

    seq_from = np.random.normal(size=(latent_dim,))
    seq_from = m_from + std_from * seq_from

    #print(decode(seq_from))
    log.append([decode(seq_from)])

In [ ]:
log

In [ ]:
df_log = process_log(log, ocel, ocpn, '../src/data/VAE_generated/DS4_process_sampled_function.csv')

In [ ]:
df_log

In [ ]:
object_types =  ["Payment application","Control summary","Entitlement application","Geo parcel document","Inspection","Reference alignment"]
parameters = {"obj_names": object_types,
              "val_names": [],
              "act_name": "event_activity",
              "time_name": "event_timestamp",
              "sep": ","}
ocel_gen = ocel_import_factory_csv.apply(file_path='../src/data/VAE_generated/DS4_process_sampled_function.csv', parameters=parameters)

# OCPN Model

In [ ]:
generalization = VAE_generalization(ocel_gen, ocpn)

# Happy Path Order

In [ ]:
happy_path__ocel = get_happy_path_log(filename)

In [ ]:
happy_path_ocpn = ocpn_discovery_factory.apply(happy_path__ocel, parameters={"debug": False})

In [ ]:
happy_path__ocel = get_happy_path_log(filename)
generalization = VAE_generalization(ocel_gen, happy_path_ocpn)

# Flower Model Order

In [ ]:
filename = "../src/data/jsonocel/DS4.jsonocel"
ots =  ["Payment application","Control summary","Entitlement application","Geo parcel document","Inspection","Reference alignment"]
flower_ocpn = create_flower_model(filename,ots)

In [ ]:
generalization = VAE_generalization(ocel_gen, flower_ocpn)

# Variant OCPN

In [ ]:
import pickle

In [ ]:
with open("../src/data/csv/DS4_variant_ocpn.pickle", "rb") as file:
    variant_ocpn = pickle.load(file)

In [ ]:
for transition in variant_ocpn.transitions:
    split_string = transition.name.split("_")
    transition.name = split_string[0]

In [ ]:
generalization = VAE_generalization(ocel_gen, variant_ocpn)